In [1]:
import glob
import subprocess
import os
import time
import pdb
from git import Repo
from numpy import linalg as LA
import numpy as np
import networkx as nx
from lxml import etree
from commit_graph import initialize_repo, get_contents
from run_srcml import transform_dir, transform_src_to_tree
from patch_parser import PatchParser
from detect_change import get_changed_functions
from write_graph_to_dot import write_G_to_dot_with_pr

In [64]:
ns = {'srcml': 'http://www.srcML.org/srcML/src', 'pos': 'http://www.srcML.org/srcML/position'}

def handle_function(func_node):
    """Given a <function> node, 
    return function name and function range (start & end lineno)"""
    
    name_node = func_node.find('srcml:name', ns)
    func_name, start_line = handle_name(name_node)
    if not func_name or not start_line:
        print('Function name/start not found!') # very unlikely to happen
        return None, None, None
    
    block_node = func_node.find('srcml:block', ns)
    if not block_node:
        try:
            block_node = func_node.xpath('./following-sibling::srcml:block', namespaces=ns)[0]
        except:
            print("More edge cases for block node! (in function {})".format(func_name))
            return func_name, None, None
    try:
        pos_node = block_node.find('pos:position', ns)
        end_line = int(pos_node.attrib['{http://www.srcML.org/srcML/position}line'])
    except:
        print("Block node doesn't have position node inside!")
        return func_name, None, None
    
    return func_name, start_line, end_line

def handle_name(name_node):
    """Given an <name> node, 
    return its text content and position (line)"""
    text, line = None, None
    if name_node:
        text = name_node.text
        line = int(name_node.attrib['{http://www.srcML.org/srcML/position}line'])
    return text, line

class NotFunctionCallError(Exception):
    """Raise for false positive <call> nodes"""

def handle_call(call_node):
    """Given an <call> node, return function name being called
    
    Throws NotFunctionCallException
    
    Case 1: casting function pointer is not function call
        Example: tmp.sa_handler = (void (*)(int)) handler;
        
    Case 2: function call from struct variable
        Example: tty->write(tty)
        
    """
    name_node = call_node.find('srcml:name', ns) 
    if not name_node:
        # Case 1
        raise NotFunctionCallError()
    callee_name = name_node.text
    if not callee_name:
        # Case 2
        callee_name = name_node[-1].text
    return callee_name

def remove_edges_of_node(G, n):
    """Remove all edges of n, but keep the node itself in the graph
    
    >>> G3 = nx.DiGraph()
    >>> G3.add_path([0, 1, 2, 3, 4])
    >>> remove_edges_of_node(G3, 2)
    >>> G3.nodes()
    [0, 1, 2, 3, 4]
    >>> G3.edges()
    [(0, 1), (3, 4)]
    
    """
    try:
        nbrs = G.succ[n]
    except KeyError: # NetworkXError if not in self
        # raise NetworkXError("The node %s is not in the digraph."%(n, ))
        print("The node %s is not in the digraph."%(n, ))
        return 
    for u in nbrs:
        del G.pred[u][n]
    G.succ[n] = {}
    for u in G.pred[n]:
        del G.succ[u][n]
    G.pred[n] = {}
    
def get_func_ranges_c(root):
    func_ranges, func_names = [], []
    for func_node in root.findall('./srcml:function', namespaces=ns):
        
        func_name, start_line, end_line = handle_function(func_node)
        if not (func_name and start_line and end_line):
            continue
        
        func_ranges.append([start_line, end_line])
        func_names.append(func_name)
    return func_names, func_ranges
    
def update_call_graph(roots, change_info, G):
    for func_name in change_info:
        if func_name in G:
            remove_edges_of_node(G, func_name)
            G.node[func_name]['num_lines'] += change_info[func_name]
        
    # here roots should be constructed from new commit
    # info of new functions is added to change_info
    build_call_graph(roots, change_info, G)
        

def build_call_graph(roots, change_info, G=None):
    if G == None:
        G = nx.DiGraph()
        
    func_to_file = {}
    for root in roots:
        # print('------ ' + root.attrib['filename'] + ' ------')
        
        for func_node in root.findall('./srcml:function', namespaces=ns):                
            
            caller_name, start_line, end_line = handle_function(func_node)
            if not caller_name:
                continue
                
            if start_line and end_line:
                num_lines = end_line - start_line + 1
            else:
                # default num_lines is 1
                num_lines = 1
                
            if caller_name not in G: # this is new function
                change_info[caller_name] = num_lines
                G.add_node(caller_name, {'num_lines': num_lines, 'defined': True}) 
            elif not G.node[caller_name]['defined']: # already in G, but haven't seen definition 
                G.node[caller_name]['defined'] = True
                G.node[caller_name]['num_lines'] = num_lines
                
            func_to_file[caller_name] = root.attrib['filename']
            
            # handle all function calls
            for call_node in func_node.xpath('.//srcml:call', namespaces=ns):
                
                try:
                    callee_name = handle_call(call_node)
                except NotFunctionCallError:
                    continue
                except:
                    print("Callee name not found! (in function {})".format(caller_name))
                    continue
                
                if callee_name not in G:
                    G.add_node(callee_name, {'num_lines': 1, 'defined': False})
                G.add_edge(caller_name, callee_name)
                
    return G, func_to_file


def devrank(G, count_self=False, alpha=0.85, epsilon=1e-5, max_iters=300):
    ni = {}
    for i, u in enumerate(G):
        ni[u] = i
        
    sizes = {}
    universe_size = 0
    for u in G:
        sizes[u] = G.node[u]['num_lines']
        universe_size += sizes[u]
        
    num_nodes = len(G.nodes())
    P = np.zeros([num_nodes, num_nodes])
    
    for u in G:
        num_out_edges = len(G[u])
        if num_out_edges == 0:
            P[:, ni[u]] = 1 / num_nodes
        else:
            total_out_sizes = 0
            for v in G[u]:
                total_out_sizes += sizes[v]
            if count_self:
                total_out_sizes += sizes[u]
                P[ni[u], ni[u]] = sizes[u] / total_out_sizes
            for v in G[u]:
                P[ni[v], ni[u]] = sizes[v] / total_out_sizes
            
    p = np.empty(num_nodes)
    for u in G:
        p[ni[u]] = sizes[u] / universe_size

    v = np.ones(num_nodes) / num_nodes
        
    for i in range(max_iters):
        new_v = alpha * np.dot(P, v) + (1 - alpha) * p
        assert(new_v.shape == (num_nodes,))
        delta = new_v - v
        if LA.norm(delta) < epsilon:
            break
        v = new_v
        
    pr = {}
    for u in G:
        pr[u] = v[ni[u]]
    
    return pr

def draw_call_graph(G, pr):
    pr = pagerank(G.reverse())
    write_G_to_dot_with_pr(G, pr, 'linux_test.dot', header_lines=['nodesep=1.0;\n'])
    subprocess.call('unflatten -l 8 -f -o unflattened_linux_test.dot linux_test.dot', shell=True)
    subprocess.call('dot -Tsvg unflattened_linux_test.dot -o unflattened_linux_test.svg', shell=True)
    


In [4]:
c_roots = []
for xml in glob.glob('linux-kernel-xml/kernel/*.c.xml'):
    tree = etree.parse(xml)
    c_roots.append(tree.getroot())
G, func_to_file = build_call_graph(c_roots, {})
print("Number of nodes: {}".format(len(G.nodes())))
print("Number of edges: {}".format(len(G.edges())))
print("Number of connected components: {}".format(nx.number_weakly_connected_components(G)))

/Users/Beaver/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:33: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
/Users/Beaver/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
/Users/Beaver/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:41: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.


Number of nodes: 5130
Number of edges: 13514
Number of connected components: 172


In [5]:
pr = devrank(G, alpha=0.01)
sorted_pr = sorted(pr.items(), key=lambda x: x[1])
# draw_call_graph(G)
[(x[0], x[1], func_to_file[x[0]] if x[0] in func_to_file else 'Unknown') for x in reversed(sorted_pr[-20:])]

[('copy_process', 0.0053585420494072821, 'linux-kernel/kernel/fork.c'),
 ('audit_receive_msg', 0.0038208782997065272, 'linux-kernel/kernel/audit.c'),
 ('sched_init', 0.003207014716320017, 'linux-kernel/kernel/sched.c'),
 ('audit_filter_rules',
  0.0030091465021866255,
  'linux-kernel/kernel/auditsc.c'),
 ('audit_log_exit', 0.0029762928232642616, 'linux-kernel/kernel/auditsc.c'),
 ('futex_requeue', 0.0029456188527405376, 'linux-kernel/kernel/futex.c'),
 ('get_signal_to_deliver',
  0.0023601726701005864,
  'linux-kernel/kernel/signal.c'),
 ('wait_task_zombie', 0.002336110733253982, 'linux-kernel/kernel/exit.c'),
 ('generate_sched_domains',
  0.002308769373404837,
  'linux-kernel/kernel/cpuset.c'),
 ('rcu_torture_init',
  0.0023046420441189845,
  'linux-kernel/kernel/rcutorture.c'),
 ('load_balance', 0.0021772356794441598, 'linux-kernel/kernel/sched.c'),
 ('posix_cpu_timer_set',
  0.0021430203058164692,
  'linux-kernel/kernel/posix-cpu-timers.c'),
 ('kgdb_handle_exception',
  0.0020964021

In [6]:
class RepoAnalyzer():
    
    def __init__(self, repo_path):
        self.repo_path = repo_path
        self.repo = initialize_repo(repo_path)
        self.git = self.repo.git # directly using git
        self.git.checkout('v2.6.32', '-f')
        self.commits = list(self.repo.iter_commits()) # first element is the latest commit
        self.G = nx.DiGraph()
        self.exts = ('.c',)
        self.history = {}
        self.share = {}
        self.patch_parser = PatchParser()
        self.EMPTY_TREE_SHA = "4b825dc642cb6eb9a060e54bf8d69288fbee4904"

        
    def evolve(self, num_commits, verbose=False):
        # handle first commit
        
        """
        first_commit = self.commits[-1]
        sha = first_commit.hexsha
        self.git.checkout(sha, '-f')
        xml_output_path = self.repo_path + "-xml-" + sha[:6]
        transform_dir(self.repo_path, xml_output_path, self.exts)
        roots = []
        for ext in self.exts:
            for xml_file in glob.glob(xml_output_path + "/**/*.xml"):
                roots.append(etree.parse(xml_file).getroot())
                
        build_call_graph(roots, {}, G=self.G)
        """
        self.G = nx.DiGraph()
        self.history = {}
        start = time.time()
        counter = 1
        for commit in reversed(self.commits[-num_commits:]):
            sha = commit.hexsha
            roots = []
            self.history[sha] = {}
            print('------  No.{} {} ------'.format(counter, sha))
            if counter % 20 == 0:
                print('-------- Used time: {} --------'.format(time.time() - start))
            
            if not commit.parents:
                diff_index = commit.diff(self.EMPTY_TREE_SHA, create_patch=True, R=True)
            else:
                last_commit = commit.parents[0]
                diff_index = commit.diff(last_commit, create_patch=True, R=True)
            
            for diff in diff_index:
                if diff.new_file:
                    diff.change_type = 'A'
                elif diff.deleted_file:
                    diff.change_type = 'D'
                elif diff.renamed: 
                    diff.change_type = 'R'
                elif diff.a_blob and diff.b_blob and diff.a_blob != diff.b_blob:
                    diff.change_type = 'M'
                else:
                    diff.change_type = 'U'
                    print("Take a look at this commit!!! {}".format(sha))
                    # raise Exception('Non-existent Change Type!')
                    continue
            
            if verbose:
                print("{}:{}".format(sha, " ".join([diff.change_type for diff in diff_index])))
            
            for diff in diff_index:
                if diff.change_type == 'U':
                    continue
                    
                if diff.change_type == 'A':
                    fname = diff.b_blob.path
                    if self.fname_filter(fname):
                        file_contents = get_contents(self.repo, commit, fname)
                        root = transform_src_to_tree(file_contents)
                        if not root:
                            continue
                        roots.append(root)
                    
                elif diff.change_type == 'D':
                    fname = diff.a_blob.path
                    if self.fname_filter(fname):
                        additions, deletions = self.parse_patch(diff.diff)
                        if not (additions and deletions):
                            continue
                            
                        file_contents = get_contents(self.repo, last_commit, fname)
                        root = transform_src_to_tree(file_contents)
                        if not root:
                            continue
                            
                        func_names, func_ranges = get_func_ranges_c(root)
                        change_info = get_changed_functions(func_names, func_ranges, additions, deletions)
                        for func_name in change_info:
                            self.history[sha][func_name] = change_info[func_name]
                        
                        # no roots.append for change_type 'D'
                        
                elif diff.change_type == 'R':
                    new_fname = diff.rename_to
                    old_fname = diff.rename_from
                    if self.fname_filter(new_fname) or self.fname_filter(old_fname):
                        additions, deletions = self.parse_patch(diff.diff)
                        if not (additions and deletions):
                            continue
                            
                        old_file_contents = get_contents(self.repo, last_commit, old_fname)
                        old_root = transform_src_to_tree(old_file_contents)
                        if not old_root:
                            continue
                        
                        func_names, func_ranges = get_func_ranges_c(old_root)
                        change_info = get_changed_functions(func_names, func_ranges, additions, deletions)
                        for func_name in change_info:
                            self.history[sha][func_name] = change_info[func_name]
                            
                        new_file_contents = get_contents(self.repo, commit, new_fname)
                        new_root = transform_src_to_tree(new_file_contents)
                        if not new_root:
                            continue
                            
                        roots.append(new_root)
                else:
                    fname = diff.b_blob.path
                    if self.fname_filter(fname):
                        additions, deletions = self.parse_patch(diff.diff)
                        if not (additions and deletions):
                            continue
                            
                        old_file_contents = get_contents(self.repo, last_commit, fname)
                        old_root = transform_src_to_tree(old_file_contents)
                        if not old_root:
                            continue
                        
                        func_names, func_ranges = get_func_ranges_c(old_root)
                        change_info = get_changed_functions(func_names, func_ranges, additions, deletions)
                        for func_name in change_info:
                            self.history[sha][func_name] = change_info[func_name]
                            
                        new_file_contents = get_contents(self.repo, commit, fname)
                        new_root = transform_src_to_tree(new_file_contents)
                        if not new_root:
                            continue
                            
                        roots.append(new_root)
                        
            update_call_graph(roots, self.history[sha], self.G)
            counter += 1
                        
                        
                        
    def parse_patch(self, patch):
        additions, deletions = None, None
        try:
            additions, deletions = self.patch_parser.parse(patch.decode("utf-8"))
        except UnicodeDecodeError:
            print("UnicodeDecodeError in function parse_patch!")
        except:
            pdb.set_trace()
            print("Unknown error in function parse_patch!")
        return additions, deletions
                    
                    
        
    def fname_filter(self, fname):
        for ext in self.exts:
            if not fname.endswith(ext):
                return False
        return True
        
        
    def update_shares(self):
        pr = devrank(self.G, alpha=0.1)
        for sha in self.history:
            self.share[sha] = 0
            for func_name in self.history[sha]:
                self.share[sha] = (self.history[sha][func_name] / self.G.node[func_name]['num_lines']) * pr[func_name]    
        
    def top_commits(n):
        pass

            
        

In [7]:
analyzer = RepoAnalyzer('linux-kernel')

In [65]:
import warnings
warnings.simplefilter('ignore')
analyzer.evolve(10)

------  No.1 9417d4148f0ddc5ee2cc1114ce97c71c5e4cb4b7 ------
------  No.2 d62bd5409667a8d2445942415a626947a1befd24 ------
------  No.3 9b7e85e9f4c289c98d8d8c0389fd6f79844a6f5f ------
------  No.4 eb619203065fb78cb27270be426853057d6f472a ------
UnicodeDecodeError in function parse_patch!
------  No.5 5b3842b9350f4c3b1898eb1ef5e5c4f45b8f889e ------
UnicodeEncodeError in transform_src_to_tree!
------  No.6 9a29bd96634bc105476885ee82fb31992e4b89b5 ------
UnicodeEncodeError in transform_src_to_tree!
------  No.7 ffc26fa924e2eca8ca5be2108b52b97269a6e512 ------
UnicodeEncodeError in transform_src_to_tree!
------  No.8 d204cc8e2a8629d2a44fd5d9bb9a4b54cf46556c ------
UnicodeEncodeError in transform_src_to_tree!
Block node doesn't have position node inside!
------  No.9 fe3eb15b85f38d60574a10860ed7e30f2db743e9 ------
------  No.10 c9da70101a8cee9027388191535c557448301779 ------


In [ ]:
analyzer.history

In [ ]:
[(n, analyzer.G.node[n]['num_lines']) for n in analyzer.G.nodes()]

In [ ]:
analyzer.G.node['write_pipe']['num_lines']

In [ ]:
analyzer.update_shares()

In [ ]:
sorted(analyzer.share.items(), key=lambda x: x[1])

In [58]:
def transform_node_to_src(node, s=None):
    if s == None:
        s = ""
    if node.text:
        s += node.text
    for child in node:
        s = transform_node_to_src(child, s)
    if node.tail:
        s += node.tail
    return s